# 🚀 VoiceAid Health — Akan ASR: Fine-Tuning

This notebook trains (fine-tunes) your custom model (`dennis-9/whisper-small_Akan_non_standardspeech`) on your **8,084 speech-impaired Akan audio dataset** to improve accuracy.

**Before running:**
- Enable GPU: **Runtime → Change runtime type → T4 GPU** (or A100/L4 if you have Colab Pro)
- Ensure your dataset and audio are in Google Drive.

In [ ]:
# Cell 1: Install Dependencies
!pip install -q transformers datasets evaluate jiwer accelerate librosa soundfile torchaudio huggingface_hub
print('✅ Dependencies installed!')

In [ ]:
# Cell 2: Login to Hugging Face
# You need to log in to upload the newly trained model to your account.
# Get your Write token from: https://huggingface.co/settings/tokens
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Cell 3: Mount Google Drive & Set Paths
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')
print('✅ Google Drive mounted!')

# ── PATHS ───────────────────────────────────────────────────────
AUDIO_DIR     = '/content/drive/MyDrive/VoiceAid_Dataset/audio/audios' # based on your previous evaluation
DATASET_EXCEL = '/content/drive/MyDrive/VoiceAid_Dataset/akan_dataset.csv'
# ────────────────────────────────────────────────────────────────

df = pd.read_csv(DATASET_EXCEL)
print(f'✅ Dataset loaded: {len(df)} rows')

In [ ]:
# Cell 4: Prepare Hugging Face Dataset
from datasets import Dataset, Audio

print("🔍 Verifying audio files...")
df['audio_path'] = df['file_name'].apply(lambda x: os.path.join(AUDIO_DIR, str(x)))

# Only keep rows where the audio file actually exists
df_filtered = df[df['audio_path'].apply(os.path.exists)].copy()
print(f"Found {len(df_filtered)} valid audio files out of {len(df)}")

df_filtered = df_filtered[['audio_path', 'text']]
df_filtered = df_filtered.rename(columns={'audio_path': 'audio'})

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df_filtered, preserve_index=False)

# Cast the audio column to the Audio feature type at 16kHz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Split into Train (90%) and Test/Validation (10%)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print("\n✅ Dataset prepared:")
print(dataset)

In [ ]:
# Cell 5: Load Processor & Tokenizer
from transformers import WhisperProcessor

MODEL_ID = "dennis-9/whisper-small_Akan_non_standardspeech"
print(f"📥 Loading processor from {MODEL_ID}...")
processor = WhisperProcessor.from_pretrained(MODEL_ID)
print("✅ Processor loaded!")

In [ ]:
# Cell 6: Preprocess Audio & Text (Feature Extraction)
print("⏳ Processing audio into spectrograms (this will take a while)...")

def prepare_dataset(batch):
    # Load and resample audio data
    audio = batch["audio"]

    # Compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Encode target text to label ids
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

# Map the function to the dataset (removes original columns to save RAM)
tokenized_dataset = dataset.map(
    prepare_dataset,
    remove_columns=dataset.column_names["train"],
    num_proc=4 # Uses multiple CPU cores for speed
)
print("✅ Preprocessing complete!")

In [ ]:
# Cell 7: Define Data Collator for Padding
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they have to be of different lengths and need different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # Pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        # If boss token is appended in previous tokenization step,
        # cut boss token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
print("✅ Collator ready!")

In [ ]:
# Cell 8: Define Metrics (WER)
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # We do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
# Cell 9: Load the Base Model for Training
from transformers import WhisperForConditionalGeneration

print(f"📥 Loading model for training: {MODEL_ID}...")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_ID)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
print("✅ Model loaded and ready to train!")

In [ ]:
# Cell 10: Set Training Arguments & Start Training!
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# We will append "_v2" to your repository name to distinguish it.
YOUR_HF_USERNAME = "dennis-9" # Make sure this matches your HF account
REPO_NAME = f"{MODEL_ID.split('/')[1]}_v2"

training_args = Seq2SeqTrainingArguments(
    output_dir=REPO_NAME,     # The directory where checkpoints are saved
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # Increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,           # Number of training steps (adjust based on how long it takes)
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor,
)

print("🚀 Starting Training... Grab a coffee, this will take hours depending on GPU!")
trainer.train()

In [ ]:
# Cell 11: Push Final Model to Hugging Face
print("📤 Pushing model to Hugging Face Hub...")
trainer.push_to_hub(commit_message="Training complete on Colab")
print(f"✅ Successfully uploaded to: https://huggingface.co/{YOUR_HF_USERNAME}/{REPO_NAME}")